In [7]:
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
import nltk
import requests
from io import StringIO
import csv

nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('punkt_tab', quiet=True)

True

In [4]:
# Load Dataset
df = pd.read_csv('/content/ulasan_aplikasi_grab.csv')
df.head()

,content
0,apk nya bagus si banyak promo nya harga go rid...
1,"Halo grab, hari ini saya mau buka aplikasi, ke..."
2,"sbg pengguna grab dari lama,khusus nya grabfoo..."
3,Aplikasinya jd lambat. Entah iklan videonya yg...
4,"Saya driver dan pengguna, dan saya mengedukasi..."


In [5]:
# Data Cleaning and Preprocessing
df.dropna(inplace=True)
clean_df = df

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip(' ')
    return text

def casefoldingText(text):
    return text.lower()

def tokenizingText(text):
    return word_tokenize(text)

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = [txt for txt in text if txt not in listStopwords]
    return filtered

def download_slangwords(url):
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()
    slang_dict = {}
    for line in lines:
        parts = line.split()
        if len(parts) == 2:
            slang_dict[parts[0].lower()] = parts[1]
    return slang_dict

url = 'https://raw.githubusercontent.com/louisowen6/NLP_bahasa_resources/master/combined_slang_words.txt'
slangwords = download_slangwords(url)

def fix_slangwords(text):
    words = text.split()
    fixed_words = [slangwords.get(word.lower(), word) for word in words]
    return ' '.join(fixed_words)

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

clean_df['text_clean'] = clean_df['content'].apply(cleaningText)
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

In [8]:
# Load Lexicons
lexicon_positive = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

lexicon_negative = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch negative lexicon data")

In [9]:
# Sentiment Analysis
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text.split():
        if word in lexicon_positive:
            score += lexicon_positive[word]
    for word in text.split():
        if word in lexicon_negative:
            score += lexicon_negative[word]

    if score > 0:
        polarity = 'positive'
    elif score < 0:
        polarity = 'negative'
    else:
        polarity = 'neutral'
    return score, polarity

results = clean_df['text_akhir'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
negative    51495
positive    49587
neutral     11418
Name: count, dtype: int64


In [10]:
# Word2Vec Training
from gensim.models import Word2Vec
texts = clean_df['text_akhir'].tolist()
def tokenize_text(text):
    return text.split()

tokenized_texts = [tokenize_text(text) for text in clean_df['text_akhir']]

model = Word2Vec(vector_size=300, window=5, min_count=5, workers=4)
model.build_vocab(tokenized_texts)
print(f"Vocabulary size: {len(model.wv)}")
model.train(tokenized_texts, total_examples=model.corpus_count, epochs=10)
model.save("word2vec_model.model")

Vocabulary size: 11143


In [11]:
df.head()

,content,text_clean,text_casefoldingText,text_slangwords,text_tokenizingText,text_stopword,text_akhir,polarity_score,polarity
0,apk nya bagus si banyak promo nya harga go rid...,apk nya bagus si banyak promo nya harga go rid...,apk nya bagus si banyak promo nya harga go rid...,apk nya bagus si banyak promo nya harga go rid...,"[apk, nya, bagus, si, banyak, promo, nya, harg...","[apk, bagus, si, promo, harga, go, ride, jg, m...",apk bagus si promo harga go ride jg murah tiba...,6,positive
1,"Halo grab, hari ini saya mau buka aplikasi, ke...",Halo grab hari ini saya mau buka aplikasi kema...,halo grab hari ini saya mau buka aplikasi kema...,halo grab hari ini saya mau buka aplikasi kema...,"[halo, grab, hari, ini, saya, mau, buka, aplik...","[halo, grab, buka, aplikasi, kemarin, fine², a...",halo grab buka aplikasi kemarin fine² aja buka...,-12,negative
2,"sbg pengguna grab dari lama,khusus nya grabfoo...",sbg pengguna grab dari lamakhusus nya grabfood...,sbg pengguna grab dari lamakhusus nya grabfood...,sbg pengguna grab dari lamakhusus nya grabfood...,"[sbg, pengguna, grab, dari, lamakhusus, nya, g...","[sbg, pengguna, grab, lamakhusus, grabfoodsy, ...",sbg pengguna grab lamakhusus grabfoodsy yg tin...,11,positive
3,Aplikasinya jd lambat. Entah iklan videonya yg...,Aplikasinya jd lambat Entah iklan videonya yg ...,aplikasinya jd lambat entah iklan videonya yg ...,aplikasinya jd lambat entah iklan videonya yg ...,"[aplikasinya, jd, lambat, entah, iklan, videon...","[aplikasinya, jd, lambat, iklan, videonya, yg,...",aplikasinya jd lambat iklan videonya yg lambat...,-6,negative
4,"Saya driver dan pengguna, dan saya mengedukasi...",Saya driver dan pengguna dan saya mengedukasi ...,saya driver dan pengguna dan saya mengedukasi ...,saya driver dan pengguna dan saya mengedukasi ...,"[saya, driver, dan, pengguna, dan, saya, menge...","[driver, pengguna, mengedukasi, orang, sy, men...",driver pengguna mengedukasi orang sy mengunaka...,-13,negative


**Pelatihan: LSTM, Ekstraksi Fitur: Word2Vec, Pembagian Data: 80/20**

In [12]:
# Preparing data for LSTM
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

model = Word2Vec.load("word2vec_model.model")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_df['text_akhir'])
sequences = tokenizer.texts_to_sequences(clean_df['text_akhir'])
word_index = tokenizer.word_index

vocab_size = len(word_index) + 1
embedding_dim = model.vector_size
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if word in model.wv:
        embedding_matrix[i] = model.wv[word]

print("Embedding matrix sample:")
print(embedding_matrix[:5])


X_sequences = tokenizer.texts_to_sequences(clean_df['text_akhir'])
max_length = max(len(seq) for seq in X_sequences)
X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(clean_df['polarity'])

X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

Embedding matrix sample:
[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [-0.08723122  0.29806438 -0.01226517 ... -0.78507704  0.27602324
  -0.17417508]
 [-0.63919884  0.43697816 -0.23991698 ...  0.2326116   0.28089514
   0.27519292]
 [-0.47750476  0.21868654  0.49903256 ...  0.2784453   0.89572728
   0.15118329]
 [-0.92413861  0.09382435 -0.464836   ...  0.23102507 -0.41087213
  -0.22024027]]


In [13]:
# Skema LSTM: Pelatihan LSTM, Ekstraksi Fitur Word2Vec, Pembagian Data 80/20

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Model LSTM
lstm_model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              weights=[embedding_matrix],
              input_length=max_length,
              trainable=False),
    Bidirectional(LSTM(128, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Compile Model
lstm_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

# Early Stopping
early_stopping_lstm = EarlyStopping(monitor='val_loss',
                                  patience=8,
                                  restore_best_weights=True)

# Training Model
history_lstm = lstm_model.fit(X_train, y_train,
                            epochs=20,
                            batch_size=32,
                            validation_data=(X_test, y_test),
                            callbacks=[early_stopping_lstm])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 128s 42ms/step - accuracy: 0.7618 - loss: 0.6017 - val_accuracy: 0.8602 - val_loss: 0.3841
Epoch 2/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 138s 42ms/step - accuracy: 0.8631 - loss: 0.3823 - val_accuracy: 0.8755 - val_loss: 0.3432
Epoch 3/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 142s 42ms/step - accuracy: 0.8840 - loss: 0.3222 - val_accuracy: 0.8797 - val_loss: 0.3273
Epoch 4/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 142s 42ms/step - accuracy: 0.8960 - loss: 0.2889 - val_accuracy: 0.8902 - val_loss: 0.2977
Epoch 5/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 141s 42ms/step - accuracy: 0.9071 - loss: 0.2588 - val_accuracy: 0.8937 - val_loss: 0.3051
Epoch 6/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 143s 42ms/step - accuracy: 0.9147 - loss: 0.2363 - val_accuracy: 0.8986 - val_loss: 0.2845
Epoch 7/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 142s 42ms/step - accuracy: 0.9234 - loss: 0.2119 - val_accuracy: 0.8990 - val_loss: 0.2876
Epoch 8/20
2813/2813 ━━━━━━━━━━━━━━━━━━━━ 142s 42ms/step - accuracy: 

In [14]:
# Evaluasi Model
loss_train_lstm, accuracy_train_lstm = lstm_model.evaluate(X_train, y_train)
print(f"Training Accuracy (LSTM): {accuracy_train_lstm * 100:.2f}%")

loss_test_lstm, accuracy_test_lstm = lstm_model.evaluate(X_test, y_test)
print(f"Testing Accuracy (LSTM): {accuracy_test_lstm * 100:.2f}%")

2813/2813 ━━━━━━━━━━━━━━━━━━━━ 47s 17ms/step - accuracy: 0.9361 - loss: 0.1740
Training Accuracy (LSTM): 93.54%
704/704 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.8975 - loss: 0.2844
Testing Accuracy (LSTM): 89.86%


**Pelatihan: Random Forest dengan Word2Vec, Pembagian Data: 80/20**

In [15]:
# Skema 1: Random Forest dengan Word2Vec

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 1. Persiapan Data (menggunakan data dari cell sebelumnya)
# Embedding matrix sudah dibuat, dan data X_padded dan y_encoded juga sudah tersedia

#2. Data Splitting
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# 3. Melatih Model Random Forest
rf_model_w2v = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1) # n_jobs=-1 untuk menggunakan semua core
rf_model_w2v.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [18]:
# 4. Evaluasi Model
y_pred_train_rf_w2v = rf_model_w2v.predict(X_train)
accuracy_train_rf_w2v = accuracy_score(y_train, y_pred_train_rf_w2v)
print(f"Training Accuracy (RF with Word2Vec): {accuracy_train_rf_w2v * 100:.2f}%")


y_pred_test_rf_w2v = rf_model_w2v.predict(X_test)
accuracy_test_rf_w2v = accuracy_score(y_test, y_pred_test_rf_w2v)
print(f"Testing Accuracy (RF with Word2Vec): {accuracy_test_rf_w2v * 100:.2f}%")

Training Accuracy (RF with Word2Vec): 100.00%
Testing Accuracy (RF with Word2Vec): 61.40%


**Pelatihan: Logistic Regression dengan TF-IDF, Pembagian Data: 80/20**

In [19]:
# Skema 3: Logistic Regression dengan TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

# 1. TF-IDF Vectorization (menggunakan data dari cell sebelumnya)
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Anda bisa mengatur max_features
tfidf_matrix = tfidf_vectorizer.fit_transform(clean_df['text_akhir'])
X_tfidf_lr = tfidf_matrix.toarray()  # Convert sparse matrix to dense array


# 2. Label Encoding (menggunakan data dari cell sebelumnya)
label_encoder = LabelEncoder()
y_lr = label_encoder.fit_transform(clean_df['polarity'])


# 3. Data Splitting
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X_tfidf_lr, y_lr, test_size=0.2, random_state=42)


# 4. Model Logistic Regression
lr_model = LogisticRegression(random_state=42, solver='liblinear') #solver liblinear for better handling small datasets
lr_model.fit(X_train_lr, y_train_lr)


LogisticRegression(random_state=42, solver='liblinear')

In [20]:
# 5. Evaluasi
y_pred_train_lr = lr_model.predict(X_train_lr)
accuracy_train_lr = accuracy_score(y_train_lr, y_pred_train_lr)
print(f"Training Accuracy (Logistic Regression with TF-IDF): {accuracy_train_lr * 100:.2f}%")


y_pred_test_lr = lr_model.predict(X_test_lr)
accuracy_test_lr = accuracy_score(y_test_lr, y_pred_test_lr)
print(f"Testing Accuracy (Logistic Regression with TF-IDF): {accuracy_test_lr * 100:.2f}%")

Training Accuracy (Logistic Regression with TF-IDF): 91.96%
Testing Accuracy (Logistic Regression with TF-IDF): 90.55%


In [23]:
# Load Word2Vec Model
model_w2v = Word2Vec.load("/content/word2vec_model.model")

# Load Logistic Regression model (dengan TF-IDF)
tfidf_vectorizer_lr = TfidfVectorizer(max_features=5000)
tfidf_matrix_lr = tfidf_vectorizer_lr.fit_transform(clean_df['text_akhir'])
X_tfidf_lr = tfidf_matrix_lr.toarray()

label_encoder_lr = LabelEncoder()
y_lr = label_encoder_lr.fit_transform(clean_df['polarity'])
lr_model = LogisticRegression(random_state=42, solver='liblinear')
lr_model.fit(X_tfidf_lr, y_lr)


# Fungsi Inferensi/Testing
def get_average_word_embedding(text, model_w2v):
    """
    Menghasilkan vector embedding rata-rata dari teks menggunakan model Word2Vec.
    """
    words = text.split()
    embeddings = [model_w2v.wv[word] for word in words if word in model_w2v.wv]
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model_w2v.vector_size)  # Return zero vector if no words in vocab

def predict_sentiment(text, model_w2v, lr_model):
    """
    Fungsi untuk memprediksi sentimen dari teks input menggunakan Word2Vec dan Logistic Regression.

    Args:
      text: string, teks yang akan diprediksi sentimennya.
      model_w2v: object, model Word2Vec.
      lr_model: object, model Logistic Regression yang dilatih dengan TF-IDF.

    Returns:
      string, kelas sentimen (negatif, netral, atau positif)
    """

    # Preprocess input text
    text_clean = cleaningText(text)
    text_casefolding = casefoldingText(text_clean)
    text_slang = fix_slangwords(text_casefolding)
    text_tokenizing = tokenizingText(text_slang)
    text_stopword = filteringText(text_tokenizing)
    text_akhir = toSentence(text_stopword)

    # Create average word embedding from input text
    avg_embedding = get_average_word_embedding(text_akhir, model_w2v)

    # Use Logistic Regression model to predict sentiment class
    input_tfidf = tfidf_vectorizer_lr.transform([text_akhir]).toarray()
    prediction = lr_model.predict(input_tfidf)[0]

    if prediction == 0:
        sentiment = 'negative'
    elif prediction == 1:
        sentiment = 'neutral'
    else:
        sentiment = 'positive'

    return sentiment

In [24]:
# Melakukan Testing

if __name__ == '__main__':
    # Contoh teks untuk diuji
    example_texts = [
        "Aplikasi ini sangat membantu dan mudah digunakan!",
        "Saya sangat kecewa dengan pelayanan aplikasi ini.",
        "Lumayan saja, tidak ada yang istimewa.",
        "Saya suka sekali dengan fitur-fitur barunya.",
        "Aplikasi ini sangat buruk dan sering error!",
        "Saya merekomendasikan aplikasi ini ke teman-teman saya",
        "tidak bagus, tolong perbaiki"
    ]

    # Contoh menggunakan model Word2Vec (dengan Logistic Regression)
    print("----- Predictions using Word2Vec with Logistic Regression -----")
    for text in example_texts:
        sentiment = predict_sentiment(text, model_w2v, lr_model)
        print(f"Text: '{text}'  -->  Sentiment: {sentiment}")

----- Predictions using Word2Vec with Logistic Regression -----
Text: 'Aplikasi ini sangat membantu dan mudah digunakan!'  -->  Sentiment: negative
Text: 'Saya sangat kecewa dengan pelayanan aplikasi ini.'  -->  Sentiment: negative
Text: 'Lumayan saja, tidak ada yang istimewa.'  -->  Sentiment: negative
Text: 'Saya suka sekali dengan fitur-fitur barunya.'  -->  Sentiment: positive
Text: 'Aplikasi ini sangat buruk dan sering error!'  -->  Sentiment: negative
Text: 'Saya merekomendasikan aplikasi ini ke teman-teman saya'  -->  Sentiment: negative
Text: 'tidak bagus, tolong perbaiki'  -->  Sentiment: negative


In [25]:
 pip freeze requirements.txt

absl-py==1.4.0
accelerate==1.2.1
aiohappyeyeballs==2.4.4
aiohttp==3.11.11
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.19
albumentations==1.4.20
altair==5.5.0
annotated-types==0.7.0
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.6.0
arviz==0.20.0
astropy==6.1.7
astropy-iers-data==0.2025.1.20.0.32.27
astunparse==1.6.3
atpublic==4.1.0
attrs==24.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.16.0
backcall==0.2.0
beautifulsoup4==4.12.3
bigframes==1.33.0
bigquery-magics==0.5.0
bleach==6.2.0
blinker==1.9.0
blis==0.7.11
blosc2==3.0.0
bokeh==3.6.2
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.1
catalogue==2.0.10
certifi==2024.12.14
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.88
clarabel==0.9.0
click==8.1.8
cloudpathlib==0.20.0
cloudpickle==3.1.1
cmake==3.31.4
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6
contourpy==1.3.1
cryptography==4

In [26]:
!pip freeze > requirements.txt